### Set up pipeline with basic imports, data directories, and file basenames.

In [ ]:
import os
from benbiohelpers.DataPipelineManagement.GenomeManager import getGenomeFastaFilePath
from bpde_chromatin_analysis.helper_scripts.BPDE_DataDir import getDataDir
from mutperiodpy.helper_scripts.UsefulFileSystemFunctions import getExternalDataDirectory as getMutperiodExternalDataDirectory

hg19FastaFilePath = getGenomeFastaFilePath("hg19")
mutperiodHg19Directory = getMutperiodExternalDataDirectory()

jiangDataDirectory = os.path.join(getDataDir(), "Jiang_BPDE_damage_maps")
jiang2uM_CellularBasename = "BEAS-2B_2uM_BPDE_cell_24h"
jiang2uM_NakedBasename = "BEAS-2B_2uM_BPDE_nDNA_24h"

### Format the reads
Format the reads by combining paired-end reads, removing lone "2nd pair" reads, removing duplicates, and combining replicates.

In [ ]:
from bpde_chromatin_analysis.FormatAlignedDamageReads import formatAlignedDamageReads

jiang2uM_CellularReadsFilePaths = list()
jiang2uM_NakedReadsFilePaths = list()

for rep in ("_rep1","_rep2","_rep3"):
    jiang2uM_CellularReadsFilePaths.append(os.path.join(jiangDataDirectory, jiang2uM_CellularBasename, jiang2uM_CellularBasename+rep+".bed"))
    jiang2uM_NakedReadsFilePaths.append(os.path.join(jiangDataDirectory, jiang2uM_NakedBasename, jiang2uM_NakedBasename+rep+".bed"))

formattedJiang2uM_CellularReadsFilePaths = formatAlignedDamageReads(jiang2uM_CellularReadsFilePaths)
formattedJiang2uM_NakedReadsFilePaths = formatAlignedDamageReads(jiang2uM_NakedReadsFilePaths)

### Convert the BPDE damage-seq reads to single-nucleotide lesion positions.

In [ ]:
from bpde_chromatin_analysis.GetBDPE_DamagePositionsFromAlignedReads import getBPDE_DamagePositionsFromAlignedReads

jiang2uM_CellularDamagePosFilePaths = getBPDE_DamagePositionsFromAlignedReads(formattedJiang2uM_CellularReadsFilePaths, hg19FastaFilePath)
jiang2uM_NakedDamagePosFilePaths = getBPDE_DamagePositionsFromAlignedReads(formattedJiang2uM_NakedReadsFilePaths, hg19FastaFilePath)

### Split damage positions into guanine and non-guanine positions.

In [ ]:
from bpde_chromatin_analysis.SplitGuaninePositions import splitGuaninePositions

jiang2uM_CellularDamagePosFilePaths = splitGuaninePositions(jiang2uM_CellularDamagePosFilePaths)
jiang2uM_NakedDamagePosFilePaths = splitGuaninePositions(jiang2uM_NakedDamagePosFilePaths)

### Parse the damage position data for mutperiod input

In [ ]:
from mutperiodpy.input_parsing.ParseStandardBed import parseStandardBed

jiang2uM_CellularDamageMutperiodInputFilePaths = parseStandardBed(jiang2uM_CellularDamagePosFilePaths, hg19FastaFilePath)
jiang2uM_NakedDamageMutperiodInputFilePath = parseStandardBed(jiang2uM_NakedDamagePosFilePaths, hg19FastaFilePath)[0]

### Run the mutperiod pipeline to determine translational and rotational periodicity of the data.

In [ ]:
from mutperiodpy.RunAnalysisSuite import runAnalysisSuite, generateCustomBackground

nucleosomeMapFilePaths = list()
# nucleosomeMapFilePaths.append(os.path.join(mutperiodHg19Directory, "hg19_hybrid_nucleosome_map", "hg19_hybrid_nucleosome_map.bed"))
# nucleosomeMapFilePaths.append(os.path.join(mutperiodHg19Directory, "hg19_LCL_MNase_nuc_map_all_mappable", "hg19_LCL_MNase_nuc_map_all_mappable.bed"))
# nucleosomeMapFilePaths.append(os.path.join(mutperiodHg19Directory, "hg19_LCL_MNase_nuc_map_all_mappable_euchromatin",
#                                            "hg19_LCL_MNase_nuc_map_all_mappable_euchromatin.bed"))
# nucleosomeMapFilePaths.append(os.path.join(mutperiodHg19Directory, "hg19_LCL_MNase_nuc_map_all_mappable_heterochromatin",
#                                            "hg19_LCL_MNase_nuc_map_all_mappable_heterochromatin.bed"))
# nucleosomeMapFilePaths.append(os.path.join(mutperiodHg19Directory, "hg19_LCL_MNase_nuc_map_all_mappable_stringent_euchromatin",
#                                            "hg19_LCL_MNase_nuc_map_all_mappable_stringent_euchromatin.bed"))
# nucleosomeMapFilePaths.append(os.path.join(mutperiodHg19Directory, "hg19_NHF1_MNase_nuc_map_mappable", "hg19_NHF1_MNase_nuc_map_all_mappable.bed"))
nucleosomeMapFilePaths.append(os.path.join(mutperiodHg19Directory, "hg19_CTCF_known_all", "hg19_CTCF_known_all.bed"))

nucleosomeMapNames = [os.path.basename(nucleosomeMapFilePath).rsplit('.', 1)[0] for nucleosomeMapFilePath in nucleosomeMapFilePaths]

generateCustomBackground(os.path.dirname(jiang2uM_NakedDamageMutperiodInputFilePath), nucleosomeMapNames,
                         useSingleNucRadius = True, includeLinker = False, useNucGroupRadius = True)

runAnalysisSuite(jiang2uM_CellularDamageMutperiodInputFilePaths, nucleosomeMapNames,
                 normalizationMethod = "Custom Background", customBackgroundDir = os.path.dirname(jiang2uM_NakedDamageMutperiodInputFilePath),
                 useSingleNucRadius = True, includeLinker = False, useNucGroupRadius = True)

In [ ]:
from mutperiodpy.helper_scripts.UsefulFileSystemFunctions import DataTypeStr
from mutperiodpy.RunNucleosomeMutationAnalysis import runNucleosomeMutationAnalysis
from benbiohelpers.FileSystemHandling.DirectoryHandling import getFilesInDirectory

runNucleosomeMutationAnalysis(getFilesInDirectory(os.path.join(jiangDataDirectory, jiang2uM_CellularBasename), DataTypeStr.normNucCounts + ".tsv"),
                              outputFilePath = os.path.join(jiangDataDirectory, "BEAS-2B_2uM_BPDE_naked_normalized_periodicity_data.tsv"),
                              overridePeakPeriodicityWithExpected = False, alignStrands = True)